In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from datasets import load_dataset,DatasetDict
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data_path = "/Users/kannavsethi/Desktop/nlp-final-project/data/AI_Human.csv"
text_column = "text"
label_column = "generated_text"

raw_data = load_dataset("csv", data_files={"full": data_path})

d0 = raw_data["full"].train_test_split(test_size=0.2, seed=42, stratify_by_column=label_column)

d1 = d0["test"].train_test_split(test_size=0.5, seed=42, stratify_by_column=label_column)

dataset = DatasetDict({
    "train": d0["train"],       # 80%
    "validation": d1["train"],  # 10%
    "test": d1["test"],         # 10%
})

